In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from pathlib import Path
from datetime import datetime
import sqlite3

In [2]:
# link splinter browser to chromedriver path
browser = Browser('chrome', executable_path='../Resources/chromedriver.exe')

In [3]:
# list of card sets formated to be end of a url string
#sset = ["innistrad-crimson-vow", "ixalan", "kaldheim", "innistrad-midnight-hunt", "war-of-the-spark", "kamigawa-neon-dynasty"]
sset = ['ixalan']    # switch commits to collect a single set

In [4]:
for x in sset:                                           # loop through set list
    url =f"https://shop.tcgplayer.com/price-guide/magic/{x}"   # visit url for set by string in list
    browser.visit(url)
    browser.driver.maximize_window()                # maximixe browser to insure all data loads 
    time.sleep(7)
    req = browser.html
    soup = BeautifulSoup(req, "html")
    names= []
    title_tags = soup.find_all('a', class_ = "pdp-url")   # capture card name
    for i in title_tags:
       names.append(i.text)
    prices =[]
    price = soup.find_all('td', class_ = "tcg-table-body__cell tcg-table-body__cell--align-right")   # capture price
    count = -1
    for i in price:
        count = count +1       # account for weird way html was written and take every other value
        if  count%2 > 0 :
            prices.append(i.text)
    price_list= zip(names, prices)     # turn name and price to a df
    mkt_df = pd.DataFrame(price_list,) 
    mkt_df = mkt_df.reset_index(drop=True)
    mkt_df.columns = ['Name', 'Market Price']
    mkt_df['date']= datetime.today().strftime('%Y-%m-%d')   # add column for todays date
    
    mkt_df['Unnamed: 0']= 2       # bug to fix!!!!!!!!!!!!!
    mkt_df = mkt_df[['Unnamed: 0', 'Name', 'Market Price', 'date']]
    price =  Path("../Resources/price.db")
    con = sqlite3.connect(price)        # conect to price db
    cur = con.cursor()
    d = datetime.today().strftime('%Y-%m-%d')
    cur.execute(f"select * from '{x}_priceData' where date = '{d}'")    # set conditional to not add price data if already exists for today
    if cur.fetchone():
        print(f"duplicate prvented for {x}")  # let know if duplicated data was prevented
    else:
        for row in mkt_df.itertuples():           # else not exist write new data to db 
            insert_sql = f''' INSERT INTO '{x}_priceData' ('Unnamed: 0', Name, 'Market Price', date) VALUES ('{row[1]}',"{row[2]}",'{row[3]}','{row[4]}')'''
            cur.execute(insert_sql)
        print(f'{x} has been update')  # let know data was updated 
    #con.commit()            # save updates commit for testing
    con.close()

duplicate prvented for ixalan
